In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from time import sleep

# Event Hub connection string and configurations
event_hub_connection_string = "Endpoint=sb://capstoneevent.servicebus.windows.net/;SharedAccessKeyName=capstoneconsumer;SharedAccessKey=KmKKRAX1DDCZjTXOmU0Wg4LVP8u14oERP+AEhHoKhIw=;EntityPath=capstonehub"
event_hub_name = "capstonehub"

# Event Hub configuration dictionary
event_hubs_conf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(event_hub_connection_string),
    'eventhubs.eventHubName': event_hub_name
}

# Read data from Event Hub
df = spark.readStream \
    .format("eventhubs") \
    .options(**event_hubs_conf) \
    .load()

# Define the schema to match the JSON data
schema = StructType([
    StructField("transaction_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
    StructField("transaction_date", TimestampType(), True),
    StructField("transaction_amount", DoubleType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("payment_type", StringType(), True)
])

# Convert the binary data to JSON and parse it
df_parsed = df.selectExpr("CAST(body AS STRING) as json").select(from_json("json", schema).alias("data")).select("data.*")

# Batching logic
batch_size = 10  # Process 10 records at a time
time_interval = 20  # Wait for 20 seconds between batches

# Hardcoded inventory data
inventory_data = {
    201: {"current_stock": 40, "reorder_level": 10},
    202: {"current_stock": 60, "reorder_level": 15},
    203: {"current_stock": 80, "reorder_level": 20},
    204: {"current_stock": 50, "reorder_level": 45},
    205: {"current_stock": 30, "reorder_level": 10},
    206: {"current_stock": 35, "reorder_level": 10},
    207: {"current_stock": 140, "reorder_level": 30},
    208: {"current_stock": 180, "reorder_level": 50},
    209: {"current_stock": 85, "reorder_level": 20},
    210: {"current_stock": 65, "reorder_level": 15}
}

# Clean and transform data
def process_batch(batch_df, batch_id):
    if batch_df.count() > 0:
        # View the actual data in the current batch (print or display)
        print(f"Batch {batch_id}:")
        batch_df.show(truncate=False)  # Show the batch data in the console

        # Handle null values
        batch_df = batch_df.withColumn(
            "transaction_amount", 
            when(col("transaction_amount").isNull(), 0).otherwise(col("transaction_amount"))
        )

        # Add any custom transformations here (e.g., converting payment_type to uppercase)
        batch_df = batch_df.withColumn("payment_type", upper(col("payment_type")))

        # Summing quantities for each product across all customers
        product_sums = batch_df.groupBy("product_id").agg(sum("quantity").alias("total_quantity"))

        # Check if any product quantities
        summed_quantities = {row["product_id"]: row["total_quantity"] for row in product_sums.collect()}
        for product_id, data in inventory_data.items():
            current_stock = data["current_stock"]
            reorder_level = data["reorder_level"]
            threshold = current_stock - reorder_level
            total_quantity_sold = summed_quantities.get(product_id, 0)
            
            # Calculate remaining stock
            remaining_stock = current_stock - total_quantity_sold
            
            if total_quantity_sold >= threshold:
                print(f"Restockreq for: {product_id}")
            else:
                print(f"Product ID: {product_id} is OK {remaining_stock}")

        # Write batch to SQL database
        jdbc_url = "jdbc:sqlserver://capstoneservo.database.windows.net:1433;database=capstonedb"
        connection_properties = {
        "user": "ash@capstoneservo",
        "password": "NOTmaurya69",
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
        batch_df.write.jdbc(url=jdbc_url, table="cap.Transactions_live", mode="append", properties=connection_properties)
        print(f"Batch {batch_id} has been processed and written to the database.")

#20 sconds
query = df_parsed \
    .writeStream \
    .foreachBatch(lambda batch_df, batch_id: [process_batch(batch_df.limit(batch_size), batch_id), sleep(time_interval)]) \
    .start()

# 12 hours in milliseconds
timeout = 12 * 60 * 60 * 1000

# Await termination with a timeout
query.awaitTermination(timeout)


Batch 1:
+--------------+-----------+--------------------------+------------------+----------+--------+------------+
|transaction_id|customer_id|transaction_date          |transaction_amount|product_id|quantity|payment_type|
+--------------+-----------+--------------------------+------------------+----------+--------+------------+
|63335         |109        |2024-10-17 14:12:43.820992|2400.0            |202       |4       |Debit Card  |
+--------------+-----------+--------------------------+------------------+----------+--------+------------+

Product ID: 201 is OK 40
Product ID: 202 is OK 56
Product ID: 203 is OK 80
Product ID: 204 is OK 50
Product ID: 205 is OK 30
Product ID: 206 is OK 35
Product ID: 207 is OK 140
Product ID: 208 is OK 180
Product ID: 209 is OK 85
Product ID: 210 is OK 65
Batch 1 has been processed and written to the database.
Batch 2:
+--------------+-----------+--------------------------+------------------+----------+--------+------------+
|transaction_id|customer_

In [0]:
import requests
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

# Logic App HTTP URL
logic_app_url = "https://prod-15.eastus.logic.azure.com:443/workflows/3681de8950d349bbae283d318a0de66b/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=bkzjXIiAKNI4erwtRlRSOm_HlqmlGD0FqdPjqaGLjVE"

# Function to send HTTP request
def send_http_request(row):
    if row.transaction_amount > 1000:
        row_dict = row.asDict()
        row_dict['transaction_date'] = row_dict['transaction_date'].strftime('%Y-%m-%d %H:%M:%S')
        response = requests.post(logic_app_url, json=row_dict)
        print(f"Sent HTTP request for transaction_id: {row.transaction_id}, status code: {response.status_code}")

# Process each partition of data
def process_partition(partition):
    for row in partition:
        send_http_request(row)

# Process each batch of data
def process_batch(df, epoch_id):
    df.filter(col("transaction_amount") > 1000).foreachPartition(process_partition)

# Apply the processing function to each batch
df_parsed.writeStream \
    .foreachBatch(process_batch) \
    .start() \
    .awaitTermination()

In [0]:
import requests
from pyspark.sql.functions import col
from pyspark.sql.types import StringType

# Logic App HTTP URL
logic_app_url = "https://prod-44.eastus.logic.azure.com:443/workflows/81511de4f009418b8d6331c7fde7c982/triggers/When_a_HTTP_request_is_received/paths/invoke?api-version=2016-10-01&sp=%2Ftriggers%2FWhen_a_HTTP_request_is_received%2Frun&sv=1.0&sig=wNTRMMdWP68o5dMPcXahFqmtIN2yuhpz3CMRpHk2dIY"

# Function to send HTTP request
def send_http_request(row):
    if row.quantity > 10 or row.transaction_amount > 1000:
        row_dict = row.asDict()
        row_dict['transaction_date'] = row_dict['transaction_date'].strftime('%Y-%m-%d %H:%M:%S')
        response = requests.post(logic_app_url, json=row_dict)
        print(f"Sent HTTP request for transaction_id: {row.transaction_id}, status code: {response.status_code}")

# Process each partition of data
def process_partition(partition):
    for row in partition:
        send_http_request(row)

# Process each batch of data
def process_batch(df, epoch_id):
    df.filter((col("quantity") > 10) | (col("transaction_amount") > 1000)).foreachPartition(process_partition)

# Apply the processing function to each batch
df_parsed.writeStream \
    .foreachBatch(process_batch) \
    .start() \
    .awaitTermination()
